In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import io

In [2]:
steam_data_df = pd.read_csv("steam.csv")
steam_data_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [3]:
steam_data_df.count()

appid               27075
name                27075
release_date        27075
english             27075
developer           27075
publisher           27075
platforms           27075
required_age        27075
categories          27075
genres              27075
steamspy_tags       27075
achievements        27075
positive_ratings    27075
negative_ratings    27075
average_playtime    27075
median_playtime     27075
owners              27075
price               27075
dtype: int64

In [4]:
steam_data_df = steam_data_df.loc[:, ["appid","name", "release_date", "publisher", "genres", "positive_ratings", "negative_ratings", "average_playtime", "median_playtime", "owners", "price"]]
steam_data_df

,appid,name,release_date,publisher,genres,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,Valve,Action,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,Valve,Action,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,Valve,Action,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,Valve,Action,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,Valve,Action,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,SHEN JIAWEI,Adventure;Casual;Indie,3,0,0,0,0-20000,2.09
27071,1065570,Cyber Gun,2019-04-23,BekkerDev Studio,Action;Adventure;Indie,8,1,0,0,0-20000,1.69
27072,1065650,Super Star Blast,2019-04-24,EntwicklerX,Action;Casual;Indie,0,1,0,0,0-20000,3.99
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,Alawar Entertainment,Adventure;Casual;Indie,2,0,0,0,0-20000,5.19


In [5]:
clean_data_df= steam_data_df.drop_duplicates(keep='first', inplace=False)
clean_data_df

,appid,name,release_date,publisher,genres,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,Valve,Action,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,Valve,Action,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,Valve,Action,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,Valve,Action,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,Valve,Action,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,SHEN JIAWEI,Adventure;Casual;Indie,3,0,0,0,0-20000,2.09
27071,1065570,Cyber Gun,2019-04-23,BekkerDev Studio,Action;Adventure;Indie,8,1,0,0,0-20000,1.69
27072,1065650,Super Star Blast,2019-04-24,EntwicklerX,Action;Casual;Indie,0,1,0,0,0-20000,3.99
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,Alawar Entertainment,Adventure;Casual;Indie,2,0,0,0,0-20000,5.19


In [6]:
 clean_data_df.rename(columns={"appid": "appid",
                                        "release_date": "release_date",
                                        "publisher": "publisher",
                                        "genres": "genres",
                                        "positive_ratings": "positive_ratings",
                                        "negative_ratings": "negative_ratings",
                                        "average_playtime": "average_playtime",
                                        "median_playtime": "median_playtime",
                                        "owners": "owners",
                                        "price": "price"}, inplace=True)

In [7]:
clean_data_df.head()

,appid,name,release_date,publisher,genres,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,Valve,Action,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,Valve,Action,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,Valve,Action,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,Valve,Action,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,Valve,Action,5250,288,624,415,5000000-10000000,3.99


In [17]:
connection_string = "postgres:postgres@localhost:5432/steam_data1"
engine = create_engine(f'postgresql://{connection_string}')

In [18]:
engine.table_names()

['steam_data1']

In [19]:
clean_data_df.to_sql(name='steam_data1', con=engine, if_exists='append', index=False)

In [20]:
pd.read_sql_query("Select * from steam_data1", con=engine)

,appid,name,release_date,publisher,genres,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,Valve,Action,124534,3339,17612,317,10000000-20000000,7
1,20,Team Fortress Classic,1999-04-01,Valve,Action,3318,633,277,62,5000000-10000000,4
2,30,Day of Defeat,2003-05-01,Valve,Action,3416,398,187,34,5000000-10000000,4
3,40,Deathmatch Classic,2001-06-01,Valve,Action,1273,267,258,184,5000000-10000000,4
4,50,Half-Life: Opposing Force,1999-11-01,Valve,Action,5250,288,624,415,5000000-10000000,4
...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,SHEN JIAWEI,Adventure;Casual;Indie,3,0,0,0,0-20000,2
27071,1065570,Cyber Gun,2019-04-23,BekkerDev Studio,Action;Adventure;Indie,8,1,0,0,0-20000,2
27072,1065650,Super Star Blast,2019-04-24,EntwicklerX,Action;Casual;Indie,0,1,0,0,0-20000,4
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,Alawar Entertainment,Adventure;Casual;Indie,2,0,0,0,0-20000,5


In [21]:
engine.execute("select * from steam_data1")